In [2]:
""" @namespace Demos of solo12 step adjustment
@file
@copyright Copyright (c) 2017-2021,
           New York University and Max Planck Gesellschaft,
           License BSD-3-Clause
@example
"""
import numpy as np
import pybullet as p
from robot_properties_solo.config import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from mim_control.robot_centroidal_controller import RobotCentroidalController
from mim_control.robot_impedance_controller import RobotImpedanceController
from reactive_planners.lipm_simulator import LipmSimpulator
from reactive_planners_cpp import QuadrupedDcmReactiveStepper
import pinocchio as pin
from scipy.spatial.transform import Rotation
from bullet_utils.env import BulletEnvWithGround

np.set_printoptions(suppress=True, precision=2)
pin.switchToNumpyArray()

def zero_cnt_gain(kp, cnt_array):
    gain = np.array(kp).copy()
    for i, v in enumerate(cnt_array):
        if v == 1:
            gain[3 * i : 3 * (i + 1)] = 0.0
    return gain


def yaw(q):
    return np.array(
        Rotation.from_quat([np.array(q)[3:7]]).as_euler("xyz", degrees=False)
    )[0, 2]

data_collector = None

In [2]:
from RAI.data_collector import DataCollector

data_collector = DataCollector()


ModuleNotFoundError: No module named 'RAI'

In [2]:
# Create a robot instance. This initializes the simulator as well.
env = BulletEnvWithGround()
robot = Solo12Robot()
env.add_robot(robot)
tau = np.zeros(12)

time = 0
sim_freq = 1000  # Hz
ctrl_freq = 1000
plan_freq = 1000

p.resetDebugVisualizerCamera(1.6, 50, -35, (0.0, 0.0, 0.0))
p.setTimeStep(1.0 / sim_freq)
p.setRealTimeSimulation(0)
for ji in range(12):
    p.changeDynamics(
        robot.robotId,
        ji,
        linearDamping=0.04,
        angularDamping=0.04,
        restitution=0.0,
        lateralFriction=4.0,
        spinningFriction=0.04,
    )

In [81]:
q = np.array(Solo12Config.initial_configuration)
q[0] = 0.
q[3:7] = pin.Quaternion(pin.rpy.rpyToMatrix(0., 0., np.pi/4)).coeffs() # 
qdot = np.matrix(Solo12Config.initial_velocity).T
robot.reset_state(q, qdot)
total_mass = sum([i.mass for i in robot.pin_robot.model.inertias[1:]])
warmup = 500
kp = np.array(12 * [50.0])
kd = 12 * [5.0]
robot_config = Solo12Config()
config_file = robot_config.paths["imp_ctrl_yaml"]
solo_leg_ctrl = RobotImpedanceController(robot, config_file)
centr_controller = RobotCentroidalController(
    robot_config,
    mu=0.6,
    kc=[0, 0, 200],
    dc=[10, 10, 10],
    kb=[25, 25, 25.],
    db=[22.5, 22.5, 22.5],
    qp_penalty_lin=[1e0, 1e0, 1e6],
    qp_penalty_ang=[1e6, 1e6, 1e6],
)
is_left_leg_in_contact = True
l_min = -0.1
l_max = 0.1
w_min = -0.08
w_max = 0.2
t_min = 0.1
t_max = 1.0
l_p = 0.00  # Pelvis width
com_height = 0.25
weight = [1, 1, 5, 1000, 1000, 100000, 100000, 100000, 100000]
mid_air_foot_height = 0.05
control_period = 0.001
planner_loop = 0.010
# init poses
robot.pin_robot.framesForwardKinematics(q)
base_pose = q[:7]
front_left_foot_position = robot.pin_robot.data.oMf[
    solo_leg_ctrl.imp_ctrl_array[0].frame_end_idx].translation
front_right_foot_position = robot.pin_robot.data.oMf[
    solo_leg_ctrl.imp_ctrl_array[1].frame_end_idx].translation
hind_left_foot_position = robot.pin_robot.data.oMf[
    solo_leg_ctrl.imp_ctrl_array[2].frame_end_idx].translation
hind_right_foot_position = robot.pin_robot.data.oMf[    
    solo_leg_ctrl.imp_ctrl_array[3].frame_end_idx].translation

v_des = np.array([0.0, 0.0, 0.0])
y_des = 0.2 # Speed of the yaw angle

quadruped_dcm_reactive_stepper = QuadrupedDcmReactiveStepper()
quadruped_dcm_reactive_stepper.initialize(
    is_left_leg_in_contact,
    l_min,
    l_max,
    w_min,
    w_max,
    t_min,
    t_max,
    l_p,
    com_height,
    weight,
    mid_air_foot_height,
    control_period,
    planner_loop,
    base_pose,
    front_left_foot_position,
    front_right_foot_position,
    hind_left_foot_position,
    hind_right_foot_position,
)

quadruped_dcm_reactive_stepper.set_desired_com_velocity(v_des)
quadruped_dcm_reactive_stepper.set_polynomial_end_effector_trajectory()

x_com = [[0.0], [0.0], [com_height]]
com_des = np.array([0., 0.0])
yaw_des = yaw(q)
cnt_array = [1, 1]
control_time = 0
open_loop = True
dcm_force = np.array([0.0, 0.0, 0.0])
offset = 0.015  # foot radius
# quadruped_dcm_reactive_stepper.start()

traj_q = np.zeros((8000 + warmup, 19))

for i in range(traj_q.shape[0]):
    if i == 1000:
        v_des = np.array([0.0, 0.0, 0.])
        
    if i == 4000:
        p.applyExternalForce(robot.robotId, -1, [-200., 200., 0.], [0., 0., 0.], p.LINK_FRAME)
    
    last_qdot = qdot
    q, qdot = robot.get_state()
    robot.pin_robot.com(q, qdot)
    robot.update_pinocchio(q, qdot)
    x_com = robot.pin_robot.com(q, qdot)[0]
    xd_com = robot.pin_robot.com(q, qdot)[1]
    traj_q[i] = q
    
    com_des += v_des[:2] * 0.001
    yaw_des += y_des * 0.001

    if i == warmup:
        quadruped_dcm_reactive_stepper.start()
#     elif i > warmup :
    else:
        FL = solo_leg_ctrl.imp_ctrl_array[0]
        FR = solo_leg_ctrl.imp_ctrl_array[1]
        HL = solo_leg_ctrl.imp_ctrl_array[2]
        HR = solo_leg_ctrl.imp_ctrl_array[3]

        # Define left as front left and back right leg
        front_left_foot_position = robot.pin_robot.data.oMf[FL.frame_end_idx].translation
        front_right_foot_position = robot.pin_robot.data.oMf[FR.frame_end_idx].translation
        hind_left_foot_position = robot.pin_robot.data.oMf[HL.frame_end_idx].translation
        hind_right_foot_position = robot.pin_robot.data.oMf[HR.frame_end_idx].translation
        front_left_foot_velocity = pin.getFrameVelocity(
            robot.pin_robot.model, robot.pin_robot.data, FL.frame_end_idx, pin.LOCAL_WORLD_ALIGNED).linear
        front_right_foot_velocity = pin.getFrameVelocity(
            robot.pin_robot.model, robot.pin_robot.data, FR.frame_end_idx, pin.LOCAL_WORLD_ALIGNED).linear
        hind_left_foot_velocity = pin.getFrameVelocity(
            robot.pin_robot.model, robot.pin_robot.data, HL.frame_end_idx, pin.LOCAL_WORLD_ALIGNED).linear
        hind_right_foot_velocity = pin.getFrameVelocity(
            robot.pin_robot.model, robot.pin_robot.data, HR.frame_end_idx, pin.LOCAL_WORLD_ALIGNED).linear

        quadruped_dcm_reactive_stepper.run(
            control_time,
            front_left_foot_position,
            front_right_foot_position,
            hind_left_foot_position,
            hind_right_foot_position,
            front_left_foot_velocity,
            front_right_foot_velocity,
            hind_left_foot_velocity,
            hind_right_foot_velocity,
            x_com,
            xd_com,
            yaw(q),
            not open_loop,
        )

        x_des_local = []
        x_des_local.extend(quadruped_dcm_reactive_stepper.get_front_left_foot_position())
        x_des_local.extend(quadruped_dcm_reactive_stepper.get_front_right_foot_position())
        x_des_local.extend(quadruped_dcm_reactive_stepper.get_hind_left_foot_position())
        x_des_local.extend(quadruped_dcm_reactive_stepper.get_hind_right_foot_position())

        cnt_array = quadruped_dcm_reactive_stepper.get_contact_array()
#     else:
#         cnt_array = [1, 1, 1, 1]
#         x_des_local = np.array([ 
#              0.195,  0.147, 0.015,
#              0.195, -0.147, 0.015,
#             -0.195,  0.147, 0.015,
#             -0.195, -0.147, 0.015
#         ])

    for j in range(4):
        imp = solo_leg_ctrl.imp_ctrl_array[j]
        x_des_local[3 * j : 3 * (j + 1)] -= imp.pin_robot.data.oMf[
            imp.frame_root_idx
        ].translation
        
    w_com = centr_controller.compute_com_wrench(
        q.copy(),
        qdot.copy(),
        [com_des[0], com_des[1], com_height],
        v_des,
        pin.Quaternion(pin.rpy.rpyToMatrix(0., 0., yaw_des)).coeffs(),
        [0.0, 0.0, y_des],
    )

    F = centr_controller.compute_force_qp(q, qdot, cnt_array, w_com)

    des_vel = np.concatenate(
        (
            quadruped_dcm_reactive_stepper.get_front_left_foot_velocity(),
            quadruped_dcm_reactive_stepper.get_front_right_foot_velocity(),
            quadruped_dcm_reactive_stepper.get_hind_left_foot_velocity(),
            quadruped_dcm_reactive_stepper.get_hind_right_foot_velocity(),
        )
    )

    if cnt_array[0] == 1:
        F[3:6] = -dcm_force[:3]
        F[6:9] = -dcm_force[:3]
    else:
        F[0:3] = -dcm_force[:3]
        F[9:12] = -dcm_force[:3]

    tau = solo_leg_ctrl.return_joint_torques(
        q.copy(),
        qdot.copy(),
        zero_cnt_gain(kp, cnt_array),
        zero_cnt_gain(kd, cnt_array),
        x_des_local,
        des_vel,
        F,
    )
    control_time += 0.001

    robot.send_joint_command(tau)
    p.stepSimulation()

    if data_collector is not None:
        data_collector.add_variable(control_time, "time", "s")
        data_collector.add_vector_3d(
            quadruped_dcm_reactive_stepper.get_front_left_foot_position(), "front_left_foot_position", "m")
        data_collector.add_vector_3d(
            quadruped_dcm_reactive_stepper.get_front_right_foot_position(), "front_right_foot_position", "m")
        data_collector.add_vector_3d(
            quadruped_dcm_reactive_stepper.get_hind_left_foot_position(), "hind_left_foot_position", "m")
        data_collector.add_vector_3d(
            quadruped_dcm_reactive_stepper.get_hind_right_foot_position(), "hind_right_foot_position", "m")

quadruped_dcm_reactive_stepper.stop()

In [80]:
import time
dq = np.zeros(18)
start = time.time()
for i in range(traj_q.shape[0]):
    robot.reset_state(traj_q[i], dq)
    time.sleep(0.00036)
dur = time.time() - start

In [ ]:
from RAI.session import Session
empty_view = {
    'Empty View':  {0: []}
}
my_session = Session()
my_session.sensors_data = data_collector
my_session._prepare(empty_view)
my_session.launch()

# Writing the same setup as DGH code

In [3]:
import dynamic_graph_head as dgh
from dynamic_graph_head import ThreadHead, SimHead, SimVicon, HoldPDController

In [4]:
bullet_env = BulletEnvWithGround()

# Create a robot instance. This initializes the simulator as well.
robot = Solo12Robot()
bullet_env.add_robot(robot)

In [5]:
import pinocchio as pin
import mim_control_cpp

class CentroidalController:
    def __init__(self, head, vicon_name, mu, kp, kd, kc, dc, kb, db, qp_weights=[5e5, 5e5, 5e5, 1e6, 1e6, 1e6]):
        self.set_k(kp, kd)
        self.config = Solo12Config
        self.robot = Solo12Config.buildRobotWrapper()
        self.vicon_name = vicon_name

        self.x_com = [0.0, 0.0, 0.20]
        self.xd_com = [0.0, 0.0, 0.0]

        self.x_des = np.array([ 
             0.2, 0.142, 0.015,  0.2, -0.142,  0.015,
            -0.2, 0.142, 0.015, -0.2, -0.142,  0.015
        ])
        self.xd_des = np.array(4*[0., 0., 0.])

        self.x_ori = [0., 0., 0., 1.]
        self.x_angvel = [0., 0., 0.]
        self.cnt_array = 4*[1,]
        
        self.w_com = np.zeros(6)
        
        q_init = np.zeros(19)
        q_init[7] = 1
        self.centrl_pd_ctrl = mim_control_cpp.CentroidalPDController()
        self.centrl_pd_ctrl.initialize(2.5, np.diag(self.robot.mass(q_init)[3:6, 3:6]))

        self.force_qp = mim_control_cpp.CentroidalForceQPController()
        self.force_qp.initialize(4, mu, np.array(qp_weights))
                
        root_name = 'universe'
        endeff_names = ['FL_ANKLE', 'FR_ANKLE', 'HL_ANKLE', 'HR_ANKLE']
        self.imp_ctrls = [mim_control_cpp.ImpedanceController() for eff_name in endeff_names]
        for i, c in enumerate(self.imp_ctrls):
            c.initialize(self.robot.model, root_name, endeff_names[i])
        
        self.kc = np.array(kc)
        self.dc = np.array(dc)
        self.kb = np.array(kb)
        self.db = np.array(db)
                
        self.joint_positions = head.get_sensor('joint_positions')
        self.joint_velocities = head.get_sensor('joint_velocities')
        self.slider_positions = head.get_sensor('slider_positions')
        self.imu_gyroscope = head.get_sensor('imu_gyroscope')

    def set_k(self, kp, kd):
        self.kp = 4 * [kp, kp, kp, 0, 0, 0]
        self.kd = 4 * [kd, kd, kd, 0, 0, 0]

    def warmup(self, thread_head):
        thread_head.vicon.bias_position(self.vicon_name)
        self.zero_sliders = self.slider_positions.copy()

    def get_base(self, thread_head):
        base_pos, base_vel = thread_head.vicon.get_state(self.vicon_name)
        base_vel[3:] = self.imu_gyroscope
        return base_pos, base_vel
    
    def compute_F(self, thread_head):
        ext_cnt_array = [1., 1., 1., 1.]
        self.force_qp.run(self.w_com, self.rel_eff, ext_cnt_array)
        self.F = self.force_qp.get_forces()
    
    def run(self, thread_head):
        base_pos, base_vel = self.get_base(thread_head)

        self.q = np.hstack([base_pos, self.joint_positions])
        self.dq = np.hstack([base_vel, self.joint_velocities])
        
        self.centrl_pd_ctrl.run(
            self.kc, self.dc, self.kb, self.db,
            self.q[:3], self.x_com, self.dq[:3], self.xd_com,
            self.q[3:7], self.x_ori, self.dq[3:6], self.x_angvel
        )
        
        self.w_com = self.centrl_pd_ctrl.get_wrench()
        self.w_com[2] += 9.81 * Solo12Config.mass
                
        # distrubuting forces to the active end effectors
        pin_robot = self.robot
        pin_robot.framesForwardKinematics(self.q)
        com = self.com = pin_robot.com(self.q)
        self.rel_eff = np.array([
            pin_robot.data.oMf[i].translation - com for i in Solo12Config.end_eff_ids
        ]).reshape(-1)
  
        self.compute_F(thread_head)
        
        # passing forces to the impedance controller
        self.tau = np.zeros(18)
        for i, c in enumerate(self.imp_ctrls):
            c.run(self.q, self.dq,
                 np.array(self.kp[6*i:6*(i+1)]),
                 np.array(self.kd[6*i:6*(i+1)]),
                 1.,
                 pin.SE3(np.eye(3), np.array(self.x_des[3*i:3*(i+1)])),
                 pin.Motion(self.xd_des[3*i:3*(i+1)], np.zeros(3)),
                 pin.Force(self.F[3*i:3*(i+1)], np.zeros(3))
             )

            self.tau += c.get_torques()
                
        head.set_control('ctrl_joint_torques', self.tau[6:])

In [6]:
class ReactiveStepperController(CentroidalController):
    def __init__(self, head):
        super().__init__(
            head, 'solo12/solo12', 0.6, 
            50, 0.7, 
            [0, 0, 200], [10, 10, 10], [25, 25, 25.], [22.5, 22.5, 22.5],
            qp_weights=[1e0, 1e0, 1e6, 1e6, 1e6, 1e6]
        )
        
        base_pos, _ = self.get_base(thread_head)
        q = np.hstack([base_pos, self.joint_positions])
                
        is_left_leg_in_contact = True
        l_min = -0.1
        l_max = 0.1
        w_min = -0.08
        w_max = 0.2
        t_min = 0.2
        t_max = 1.0
        l_p = 0.00  # Pelvis width
        com_height = 0.25
        weight = [1, 1, 5, 1000, 1000, 100000, 100000, 100000, 100000]
        mid_air_foot_height = 0.05
        control_period = 0.001
        planner_loop = 0.010
        # init poses
        self.robot.framesForwardKinematics(q)
        base_pose = q[:7]
        
        front_left_foot_position = self.robot.data.oMf[
            self.config.end_eff_ids[0]].translation
        front_right_foot_position = self.robot.data.oMf[
            self.config.end_eff_ids[1]].translation
        hind_left_foot_position = self.robot.data.oMf[
            self.config.end_eff_ids[2]].translation
        hind_right_foot_position = self.robot.data.oMf[    
            self.config.end_eff_ids[3]].translation

        self.stepper = QuadrupedDcmReactiveStepper()
        self.stepper.initialize(
            is_left_leg_in_contact,
            l_min,
            l_max,
            w_min,
            w_max,
            t_min,
            t_max,
            l_p,
            com_height,
            weight,
            mid_air_foot_height,
            control_period,
            planner_loop,
            base_pose,
            front_left_foot_position,
            front_right_foot_position,
            hind_left_foot_position,
            hind_right_foot_position,
        )
        self.stepper.set_dynamical_end_effector_trajectory()
        self.stepper.set_desired_com_velocity(np.array([0.0, 0.0, 0.0]))
        
        self.x_com[2] = com_height
        
    def warmup(self, thread_head):
        super().warmup(thread_head)
        
        self.stepper.start()
        self.control_time = 0.
        
    def compute_F(self, thread_head):
        config = self.config
        robot = self.robot
        x_com, xd_com = self.robot.com(self.q, self.dq)
        
        robot.forwardKinematics(self.q, self.dq)
        robot.framesForwardKinematics(self.q)
        
        # Define left as front left and back right leg
        front_left_foot_position = robot.data.oMf[config.end_eff_ids[0]].translation
        front_right_foot_position = robot.data.oMf[config.end_eff_ids[1]].translation
        hind_left_foot_position = robot.data.oMf[config.end_eff_ids[2]].translation
        hind_right_foot_position = robot.data.oMf[config.end_eff_ids[3]].translation
        front_left_foot_velocity = pin.getFrameVelocity(
            robot.model, robot.data, config.end_eff_ids[0], pin.LOCAL_WORLD_ALIGNED).linear
        front_right_foot_velocity = pin.getFrameVelocity(
            robot.model, robot.data, config.end_eff_ids[1], pin.LOCAL_WORLD_ALIGNED).linear
        hind_left_foot_velocity = pin.getFrameVelocity(
            robot.model, robot.data, config.end_eff_ids[2], pin.LOCAL_WORLD_ALIGNED).linear
        hind_right_foot_velocity = pin.getFrameVelocity(
            robot.model, robot.data, config.end_eff_ids[3], pin.LOCAL_WORLD_ALIGNED).linear

        open_loop = True
        self.stepper.run(
            self.control_time,
            front_left_foot_position,
            front_right_foot_position,
            hind_left_foot_position,
            hind_right_foot_position,
            front_left_foot_velocity,
            front_right_foot_velocity,
            hind_left_foot_velocity,
            hind_right_foot_velocity,
            x_com,
            xd_com,
            yaw(self.q),
            not open_loop,
        )

        cnt_array = self.stepper.get_contact_array()
        self.force_qp.run(self.w_com, self.rel_eff, cnt_array)
        self.F = self.force_qp.get_forces()

#         dcm_forces = self.stepper.get_forces()
        
#         if cnt_array[0] == 1:
#             self.F[3:6] = -dcm_forces[6:9]
#             self.F[6:9] = -dcm_forces[6:9]
#         else:
#             self.F[0:3] = -dcm_forces[:3]
#             self.F[9:12] = -dcm_forces[:3]
        
        self.x_des = np.hstack([
            self.stepper.get_front_left_foot_position(),
            self.stepper.get_front_right_foot_position(),
            self.stepper.get_hind_left_foot_position(),
            self.stepper.get_hind_right_foot_position()
        ])
        
        self.dx_des = np.hstack([
            self.stepper.get_front_left_foot_velocity(),
            self.stepper.get_front_right_foot_velocity(),
            self.stepper.get_hind_left_foot_velocity(),
            self.stepper.get_hind_right_foot_velocity(),
        ])
        
        self.control_time += 0.001

In [7]:
head = SimHead(robot, vicon_name='solo12')
thread_head = ThreadHead(
    0.001, # dt.
    HoldPDController(head, 3., 0.05, True), # Safety controllers.
    head, # Heads to read / write from.
    [     # Utils.
        ('vicon', SimVicon(['solo12/solo12']))
    ], 
    bullet_env # Environment to step.
)

Hello world from websocket thread. <ThreadHead(Thread-4, initial)>


In [8]:
q, dq = np.array(Solo12Config.initial_configuration), np.array(Solo12Config.initial_velocity)
q[0] = 0.

head.reset_state(q, dq)
thread_head.sim_run(1)

thread_head.switch_controllers(thread_head.safety_controllers)
thread_head.safety_controllers[0].warmup(thread_head)

thread_head.sim_run(1000)

In [9]:
centroidal_controller = CentroidalController(head, 'solo12/solo12', 0.2, 50., 0.7,
    [100., 100., 100.], [15., 15., 15.], [25., 25., 25.], [22.5, 22.5, 22.5]
)

thread_head.switch_controllers(centroidal_controller)

In [10]:
ctrl = ReactiveStepperController(head)
thread_head.switch_controllers(ctrl)

In [10]:
thread_head.start_streaming()
thread_head.sim_run(60000)

  Not logging 'kp' as field type '<class 'list'>' is unsupported
  Not logging 'kd' as field type '<class 'list'>' is unsupported
  Not logging 'config' as field type '<class 'type'>' is unsupported
  Not logging 'robot' as field type '<class 'pinocchio.robot_wrapper.RobotWrapper'>' is unsupported
  Not logging 'vicon_name' as field type '<class 'str'>' is unsupported
  Not logging 'x_com' as field type '<class 'list'>' is unsupported
  Not logging 'xd_com' as field type '<class 'list'>' is unsupported
  Not logging 'x_ori' as field type '<class 'list'>' is unsupported
  Not logging 'x_angvel' as field type '<class 'list'>' is unsupported
  Not logging 'cnt_array' as field type '<class 'list'>' is unsupported
  Not logging 'centrl_pd_ctrl' as field type '<class 'mim_control_cpp.CentroidalPDController'>' is unsupported
  Not logging 'force_qp' as field type '<class 'mim_control_cpp.CentroidalForceQPController'>' is unsupported
  Not logging 'imp_ctrls' as field type '<class 'list'>' is 

error: Not connected to physics server.